In [26]:
import json
from sklearn.model_selection import train_test_split
from tqdm.std import tqdm

In [2]:
with open("annotation/final_simple_annotation.json", "r") as f:
    data = json.load(f)

In [3]:
new_data = data.get("text_data") + data.get("table_data")

In [4]:
train,test = train_test_split(new_data,test_size=0.2)

In [5]:
len(train)

5279

In [6]:
len(test)

1320

## statistic concept information

In [7]:
## for training set
train_concepts = set()

In [8]:
for element in tqdm(train):
    entities = element.get("numeric_entities")
    if entities:
        for entity in entities:
            train_concepts.add(entity.get("concept"))

100%|██████████| 5279/5279 [00:00<00:00, 381872.49it/s]


In [9]:
len(train_concepts)

2098

In [10]:
# for testing set
test_concepts = set()

In [11]:
for element in tqdm(test):
    entities = element.get("numeric_entities")
    if entities:
        for entity in entities:
            test_concepts.add(entity.get("concept"))

100%|██████████| 1320/1320 [00:00<00:00, 394195.89it/s]


In [12]:
len(test_concepts)

1218

In [13]:
missed_concepts = test_concepts-train_concepts

In [14]:
len(missed_concepts)

190

In [15]:
new_missed_concepts = missed_concepts.copy()

## Supplementary training set

In [16]:
temp_train = []

In [17]:
for target_concept in new_missed_concepts:
    for line in test:
        entities = line.get("numeric_entities")
        if entities:
            for entity in entities:
                if target_concept == entity.get("concept"):
                    temp_train.append(line)

In [18]:
new_train = train+temp_train

In [20]:
len(temp_train)

550

In [21]:
len(new_train)

5829

In [24]:
with open("annotation/train_data.json", "w") as f:
    json.dump(new_train, f, indent=4)

In [25]:
with open("annotation/test_data.json", "w") as f:
    json.dump(test, f, indent=4)

## total concepts

In [28]:
total_concepts = set()

In [29]:
for element in tqdm(new_data):
    entities = element.get("numeric_entities")
    if entities:
        for entity in entities:
            total_concepts.add(entity.get("concept"))

100%|██████████| 6599/6599 [00:00<00:00, 464290.47it/s]


In [30]:
len(total_concepts)

2288

## constrcut label2id data

In [31]:
labels = list(total_concepts)

bio_labels = []
for label in labels:
    bio_labels.append(f"B-{label}")
    bio_labels.append(f"I-{label}")
bio_labels.append("O")  # 最后加 'O'

In [32]:
label2id = {label: idx for idx, label in enumerate(bio_labels)}
id2label = {str(idx): label for label, idx in label2id.items()}

In [33]:
with open("annotation/BIO_data/label2id.json", "w") as f:
    json.dump(label2id, f, indent=2)

with open("annotation/BIO_data/id2label.json", "w") as f:
    json.dump(id2label, f, indent=2)